## LangChain Component

### 1. 스키마

- **텍스트**: 텍스트 인터페이스로 언어모델과 주로 상호작용.

In [ ]:
from langchain.schema import Text
# 
text_input = Text(content="안녕 오늘 도움을 받을 수 있을까?")

- **채팅 메시지**: 시스템, 사용자, AI의 채팅 메시지.

In [ ]:
from langchain.schema import SystemChatMessage, HumanChatMessage, AIChatMessage

system_message = SystemChatMessage(content="너는 내 도우미야")
user_message = HumanChatMessage(content="오늘 비가 올까?")
ai_message = AIChatMessage(content="오늘 오후에 비가 올 수 있으니 우산을 챙겨")

- **예시**: 모델 훈련 및 평가용 입력/출력 쌍.

In [ ]:
from langchain.schema import Example

example = Example(input="안녕을 영어로 어떻게 말해?", output="Hello")

- **문서**: 비구조화된 데이터 조각.

In [ ]:
from langchain.schema import Document

document = Document(page_content="This is a sample document content.", metadata={"author": "John Doe"})

### 2. 모델

- **대규모언어모델(LLMs)**

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(api_key="api-key")
# response =  괄호 안의 텍스트에 대한 응답
response = llm.predict("이 문제 푸는 방법 알려줘")

- **채팅 모델**: 채팅 형식의 모델.

In [ ]:
from langchain.chat_models import ChatOpenAI

# 모델 선언
chat_model = ChatOpenAI(model="gpt-3.5-turbo", temporature=0)
# user_message에 대한 응답
chat_response = chat_model.chat([user_message])

- **텍스트 임베딩 모델**: 텍스트를 임베딩(벡터)로 변환하는 모델.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
# 임베딩 수행할 모델 선언
embeddings_model = OpenAIEmbeddings()
# 입력한 텍스트를 임베딩
embeddings = embeddings_model.embed_text("Sample text to embed")

### 3. 프롬프트

- **프롬프트 값(PromptValue)**: 모델에 입력되는 값.

In [ ]:
from langchain.prompts import PromptValue
# prompt_value 선언
prompt_value = PromptValue(text="AI에 대해 설명해줘")

- **프롬프트 템플릿**: 다양한 입력 구성을 지원하는 템플릿.

In [ ]:
from langchain.prompts import PromptTemplate

template = PromptTemplate(input_variables=["location"], template="What is the weather like in {location}?")
prompt = template.format(location="서울")

- **출력 파서**: 모델 출력 형식을 지정하고 파싱.

In [ ]:
from langchain.output_parsers import OutputParser

class SimpleOutputParser(OutputParser):
    # 응답을 받아서 양쪽 공백을 제거하고 반환
    def parse(self, response: str):
        return response.strip()

parser = SimpleOutputParser()
#  parse에서 지정한 형식으로 반환
parsed_output = parser.parse("  응닶 내용입니다.  ")

### 4. 인덱스

- **문서 로더**: 다양한 파일 형식을 텍스트 데이터로 변환.

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader

loader = UnstructuredFileLoader(file_path="sample.pdf")
documents = loader.load()

- **텍스트 분할기**: 긴 텍스트를 여러 조각으로 나눔.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# chunk_size(각 조각의 최대 크기 여기서는 100 문자) 크기로 텍스트를 나눈다
splitter = CharacterTextSplitter(chunk_size=100)
chunks = splitter.split("긴 문장을 문단으로 나눠줘")

- **벡터 저장소**: 임베딩된 벡터를 저장하고 검색.

In [ ]:
from langchain.vectorstores import FAISS
# FAISS(Facebook AI Research에서 개발한 라이브러리)를 사용해서 벡터 저장소 생성
vector_store = FAISS()
vector_store.add_texts(["This is a sample document.", "Another document for testing."])

- **검색기**: 쿼리를 입력으로 받아 관련 문서 반환.

In [ ]:
from langchain.retrievers import SimpleRetriever

retriever = SimpleRetriever(vector_store=vector_store)
results = retriever.retrieve("query")

### 5. 체인

- **체인(Chain)**: 여러 구성 요소를 결합하여 특정 목적 수행.

In [ ]:
from langchain.chains import SimpleChain

chain = SimpleChain(prompt_template=template, model=llm, output_parser=parser)
result = chain.run(input_variables={"location": "Seoul"})

- **LLM 체인**: 프롬프트 템플릿과 모델을 조합해 사용.

In [ ]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt_template=template, model=llm)
response = llm_chain.run({"location": "Seoul"})

- **인덱스 연관 체인**: 문서와 상호작용하여 질문에 답변.

In [ ]:
from langchain.chains import IndexRelatedChain

index_chain = IndexRelatedChain(index=vector_store, model=llm)
answer = index_chain.run(query="이 내용이 의미하는게 뭐야?")

### 6. 메모리

- **채팅 메시지 히스토리**: 이전 채팅 상호작용을 기억.

In [ ]:
from langchain.memory import ChatMessageHistory
# 대화 기록을 관리
history = ChatMessageHistory()
history.add_message(user_message)
history.add_message(ai_message)

- **장기 및 단기 메모리**: 대화 데이터 저장 및 검색.

In [ ]:
from langchain.memory import LongTermMemory, ShortTermMemory

long_term_memory = LongTermMemory()
short_term_memory = ShortTermMemory()

long_term_memory.add("날씨에 대해 물어봤지")
short_term_memory.add("Assistant replied 비가온다고 답했다")

### 7. 에이전트

- **도구(Tools)**: 언어 모델이 리소스와 상호작용.

In [ ]:
from langchain.tools import WebSearchTool

web_search_tool = WebSearchTool(api_key="api-key")
search_results = web_search_tool.query("어제 날씨에 대해 알려줘")

- **툴킷(Toolkits)**: 특정 작업을 수행하는 도구 세트.

In [ ]:
from langchain.toolkits import DataAnalysisToolkit

toolkit = DataAnalysisToolkit(tools=[web_search_tool])
analysis_results = toolkit.analyze("Analyze data trends in AI research.")

- **에이전트(Agents)**: 사용자 입력을 받아 액션을 수행.

In [ ]:
from langchain.agents import BasicAgent

agent = BasicAgent(model=llm, tools=[web_search_tool])
agent_response = agent.handle_input("머신러닝에 대한 최신 논문을 찾아줘")